<img src="media/docker_logo_1.png" width="500" height="500">

In [1]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        3 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        3 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        4 months ago        1.848 kB


In [3]:
nvidia-docker run --rm nvidia/cuda:8.0-cudnn5-devel nvidia-smi

Sat Nov 12 21:09:56 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [5]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [6]:
nvidia-docker run nvidia/cuda:8.0-cudnn5-devel nvidia-smi

Sat Nov 12 21:11:30 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [8]:
docker ps -a

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS                      PORTS               NAMES
e310b48e17f9        nvidia/cuda:8.0-cudnn5-devel   "nvidia-smi"        12 seconds ago      Exited (0) 11 seconds ago                       backstabbing_engelbart


In [9]:
nvidia-docker start e310b48e17f9

e310b48e17f9


In [10]:
nvidia-docker start --help


Usage:	docker start [OPTIONS] CONTAINER [CONTAINER...]

Start one or more stopped containers

Options:
  -a, --attach               Attach STDOUT/STDERR and forward signals
      --detach-keys string   Override the key sequence for detaching a container
      --help                 Print usage
  -i, --interactive          Attach container's STDIN


In [11]:
nvidia-docker start -i e310b48e17f9

Sat Nov 12 21:16:36 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
nvidia-docker start -a e310b48e17f9

Sat Nov 12 21:17:01 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [13]:
nvidia-docker exec e310b48e17f9 echo "hello from nvidia"

Error response from daemon: Container e310b48e17f97c1103c53a3c91d008581019782cb523462c68dbbcebc4043e54 is not running


In [14]:
nvidia-docker run --interactive --tty -d e310b48e17f9 /bin/bash

Using default tag: latest
Pulling repository docker.io/library/e310b48e17f9
nvidia-docker | 2016/11/12 21:29:35 Error: image library/e310b48e17f9:latest not found


In [16]:
docker ps -a

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS                      PORTS               NAMES
e310b48e17f9        nvidia/cuda:8.0-cudnn5-devel   "nvidia-smi"        20 minutes ago      Exited (0) 14 minutes ago                       backstabbing_engelbart


In [17]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        3 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        3 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        4 months ago        1.848 kB


In [18]:
nvidia-docker run --interactive --tty -d nvidia/cuda:8.0-cudnn5-devel /bin/bash

1868fbf74e0b9e613b4a41760f2b31257c280624a3b70d4705d4cb8c4f519150


In [19]:
docker ps 

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS              PORTS               NAMES
1868fbf74e0b        nvidia/cuda:8.0-cudnn5-devel   "/bin/bash"         16 seconds ago      Up 15 seconds                           stupefied_bhabha


In [26]:
nvidia-docker exec 1868fbf74e0b ls /usr/local/cuda/extras

CUPTI
Debugger
